In [10]:
import re


In [11]:
ABBREVIATIONS = {
    "rd": "road",
    "st": "street",
    "ave": "avenue",
    "apt": "apartment",
    "nr": "near",
    "opp": "opposite",
    "hyd": "hyderabad",
    "blvd": "boulevard"
}


In [8]:
def normalize_address(address):
    # convert to lowercase
    address = address.lower()
    
    # remove special characters (like commas, dots)
    address = re.sub(r"[^\w\s]", " ", address)
    
    # remove extra spaces
    address = re.sub(r"\s+", " ", address).strip()
    
    words = address.split()
    
    new_words = []
    
    for word in words:
        if word in ABBREVIATIONS:
            new_words.append(ABBREVIATIONS[word])
        else:
            new_words.append(word)
    
    return " ".join(new_words)


In [ ]:
This function first converts the address to lowercase, removes special characters using regular expressions, removes extra spaces, and splits the address into individual words.

Then it checks each word against the ABBREVIATIONS dictionary, replaces it if a full form exists, and finally joins the words back together to return a cleaned and standardized address string.

In [12]:
address = "Flat 12B, Sai Residency, MG Rd, Hyd"

print(normalize_address(address))


flat 12b sai residency mg road hyderabad


In [6]:
def extract_components(address):
    address = normalize_address(address)
    
    # Find 6-digit PIN code
    pincode_match = re.search(r"\b\d{6}\b", address)
    pincode = pincode_match.group() if pincode_match else None
    
    # Find house/flat number (number + optional letter)
    house_match = re.search(r"\b\d+[a-zA-Z]?\b", address)
    house_number = house_match.group() if house_match else None
    
    return {
        "house_number": house_number,
        "pincode": pincode,
        "full_text": address
    }


In [ ]:
This function first calls normalize_address(address) to clean and standardize the address before extracting components.

It then uses regular expressions to find a 6-digit PIN code and a house/flat number (a number with an optional letter), and returns these along with the full cleaned address in a dictionary for structured comparison

In [7]:
address = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081"

result = extract_components(address)

print(result)


{'house_number': '12b', 'pincode': '500081', 'full_text': 'flat 12b sai residency mg road hyderabad 500081'}


In [8]:
"45 KPHB Colony, Hyd 500072"
"Plot 18A, Near Metro Station, Gachibowli"
"H.No 302, Jubilee Hills, Hyderabad"


'H.No 302, Jubilee Hills, Hyderabad'

In [9]:
addresses = [
    "45 KPHB Colony, Hyd 500072",
    "Plot 18A, Near Metro Station, Gachibowli",
    "H.No 302, Jubilee Hills, Hyderabad"
]

for addr in addresses:
    print("Input:", addr)
    print("Extracted:", extract_components(addr))
    print("-" * 50)


Input: 45 KPHB Colony, Hyd 500072
Extracted: {'house_number': '45', 'pincode': '500072', 'full_text': '45 kphb colony hyderabad 500072'}
--------------------------------------------------
Input: Plot 18A, Near Metro Station, Gachibowli
Extracted: {'house_number': '18a', 'pincode': None, 'full_text': 'plot 18a near metro station gachibowli'}
--------------------------------------------------
Input: H.No 302, Jubilee Hills, Hyderabad
Extracted: {'house_number': '302', 'pincode': None, 'full_text': 'h no 302 jubilee hills hyderabad'}
--------------------------------------------------


In [67]:
def extract_components_v3(address):
    import re
    
    address_lower = address.lower()
    components = {}
    
    # PINCODE
    pin_match = re.search(r"\b\d{6}\b", address_lower)
    if pin_match:
        components["pincode"] = pin_match.group()
    
    # HOUSE NUMBER
    house_match = re.search(r"\b\d+[a-z]?\b", address_lower)
    if house_match:
        components["house_number"] = house_match.group()
    
    # SIMPLE CITY DETECTION
    if "hyderabad" in address_lower:
        components["city"] = "hyderabad"
    
    components["full_text"] = address_lower
    
    return components


In [ ]:
pin_match = re.search(r"\b\d{6}\b", address_lower)
\d{6} → looks for exactly 6 digits
\b → ensures it matches a complete word (not part of a longer number)
So it finds a valid 6-digit PIN code like:

500081


If found:

components["pincode"] = pin_match.group()



It stores the matched PIN code inside the dictionary.
This function converts the address to lowercase and then uses regular expressions to extract structured components like the 6-digit pincode and house number (number with optional letter).
It also checks if the city name “hyderabad” is present, stores all detected components along with the full lowercase address in a dictionary, and returns it for further matching and scoring.


In [11]:
addresses = [
    "45 KPHB Colony, Hyd 500072",
    "Plot 18A, Near Metro Station, Gachibowli",
    "H.No 302, Jubilee Hills, Hyderabad",
    "3rd Cross, 5th Main, H.No 45, Banjara Hills"
]

for addr in addresses:
    print("Input:", addr)
    print("Extracted V2:", extract_components_v2(addr))
    print("-" * 50)


Input: 45 KPHB Colony, Hyd 500072
Extracted V2: {'house_number': '45', 'pincode': '500072', 'full_text': '45 kphb colony hyderabad 500072'}
--------------------------------------------------
Input: Plot 18A, Near Metro Station, Gachibowli
Extracted V2: {'house_number': '18a', 'pincode': None, 'full_text': 'plot 18a near metro station gachibowli'}
--------------------------------------------------
Input: H.No 302, Jubilee Hills, Hyderabad
Extracted V2: {'house_number': '302', 'pincode': None, 'full_text': 'h no 302 jubilee hills hyderabad'}
--------------------------------------------------
Input: 3rd Cross, 5th Main, H.No 45, Banjara Hills
Extracted V2: {'house_number': '45', 'pincode': None, 'full_text': '3rd cross 5th main h no 45 banjara hills'}
--------------------------------------------------


In [3]:
def calculate_similarity(addr1, addr2):
    comp1 = extract_components_v2(addr1)
    comp2 = extract_components_v2(addr2)
    
    # Full text similarity
    text_score = fuzz.token_sort_ratio(comp1["full_text"], comp2["full_text"])
    
    # House number match boost
    house_score = 0
    if comp1["house_number"] and comp2["house_number"]:
        if comp1["house_number"] == comp2["house_number"]:
            house_score = 20
    
    # PIN code match boost
    pin_score = 0
    if comp1["pincode"] and comp2["pincode"]:
        if comp1["pincode"] == comp2["pincode"]:
            pin_score = 30
    
    final_score = text_score + house_score + pin_score
    
    return {
        "text_similarity": text_score,
        "house_bonus": house_score,
        "pincode_bonus": pin_score,
        "final_score": final_score
    }


In [ ]:
This function compares two addresses by first extracting structured components (house number, pincode, full text) using extract_components_v2.

Then it calculates:

Text similarity using fuzz.token_sort_ratio, which compares the full address text while ignoring word order.

Adds a house number bonus (20 points) if both house numbers exist and match exactly.

Adds a pincode bonus (30 points) if both PIN codes exist and match exactly.

Finally, it adds all these scores together to create a final_score and returns a dictionary showing the breakdown of text similarity, house bonus, pincode bonus, and total score.

In [15]:
!pip install rapidfuzz


In [1]:
from rapidfuzz import fuzz


C:\Users\DELL\Desktop\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\DELL\Desktop\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [13]:
addr1 = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081"
addr2 = "12B Sai Residency MG Road Hyd 500081"

print(calculate_similarity(addr1, addr2))


{'text_similarity': 94.3820224719101, 'house_bonus': 20, 'pincode_bonus': 30, 'final_score': 144.3820224719101}


In [15]:
def decide_match(addr1, addr2, threshold=100):
    result = calculate_similarity(addr1, addr2)
    
    final_score = result["final_score"]
    
    if final_score >= threshold:
        decision = "MATCH"
    else:
        decision = "NO MATCH"
    
    return {
        "decision": decision,
        "confidence_score": round(final_score, 2),
        "details": result
    }


In [ ]:
This function makes the **final decision** based on the total similarity score.

It first calls `calculate_similarity()` to get the `final_score`, then compares that score with a predefined `threshold` (default = 100). If the score is greater than or equal to the threshold, it returns `"MATCH"`; otherwise, it returns `"NO MATCH"`, along with the rounded confidence score and detailed score breakdown.


In [16]:
addr1 = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081"
addr2 = "12B Sai Residency MG Road Hyd 500081"

print(decide_match(addr1, addr2))


{'decision': 'MATCH', 'confidence_score': 144.38, 'details': {'text_similarity': 94.3820224719101, 'house_bonus': 20, 'pincode_bonus': 30, 'final_score': 144.3820224719101}}


In [17]:
addr1 = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081"
addr2 = "Flat 99C, Lake View Apartments, Gachibowli 500032"

print(decide_match(addr1, addr2))


{'decision': 'NO MATCH', 'confidence_score': 36.17, 'details': {'text_similarity': 36.170212765957444, 'house_bonus': 0, 'pincode_bonus': 0, 'final_score': 36.170212765957444}}


In [18]:
def decide_match_v2(addr1, addr2, threshold=100):
    result = calculate_similarity(addr1, addr2)
    
    comp1 = extract_components_v2(addr1)
    comp2 = extract_components_v2(addr2)
    
    final_score = result["final_score"]
    
    # 🚨 Hard conflict rules
    
    # If both PIN codes exist and are different → NO MATCH
    if comp1["pincode"] and comp2["pincode"]:
        if comp1["pincode"] != comp2["pincode"]:
            return {
                "decision": "NO MATCH",
                "reason": "Different PIN codes",
                "confidence_score": round(final_score, 2)
            }
    
    # If both house numbers exist and are different → reduce confidence heavily
    if comp1["house_number"] and comp2["house_number"]:
        if comp1["house_number"] != comp2["house_number"]:
            final_score -= 40  # strong penalty
    
    # Final decision
    if final_score >= threshold:
        decision = "MATCH"
    else:
        decision = "NO MATCH"
    
    return {
        "decision": decision,
        "confidence_score": round(final_score, 2),
        "details": result
    }


In [ ]:
First, it calculates the similarity score and extracts structured components (PIN and house number). If both PIN codes exist and are different, it immediately returns “NO MATCH”, because different PIN codes strongly indicate different addresses.

If house numbers exist and are different, it does not reject immediately, but reduces the final score by 40 points as a strong penalty.

After applying these rules, it compares the updated score with the threshold to decide MATCH or NO MATCH, and returns the decision along with the confidence score and detailed breakdown.

In [19]:
addr1 = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081"
addr2 = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500032"

print(decide_match_v2(addr1, addr2))


{'decision': 'NO MATCH', 'reason': 'Different PIN codes', 'confidence_score': 115.74}


In [20]:
addr1 = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081"
addr2 = "Flat 99C, Sai Residency, MG Rd, Hyderabad 500081"

print(decide_match_v2(addr1, addr2))


{'decision': 'NO MATCH', 'confidence_score': 81.49, 'details': {'text_similarity': 91.48936170212765, 'house_bonus': 0, 'pincode_bonus': 30, 'final_score': 121.48936170212765}}


In [21]:
def bulk_match(query_address, address_list, threshold=100):
    matches = []
    
    for candidate in address_list:
        result = decide_match_v2(query_address, candidate, threshold)
        
        if result["decision"] == "MATCH":
            matches.append({
                "address": candidate,
                "confidence_score": result["confidence_score"]
            })
    
    # Sort by highest confidence
    matches = sorted(matches, key=lambda x: x["confidence_score"], reverse=True)
    
    return matches


In [ ]:
This function compares one query address against multiple addresses in a list and finds all matches.

It loops through each candidate address, calls `decide_match_v2()` to check if it matches the query, and if the decision is `"MATCH"`, it stores the address along with its confidence score.

After checking all addresses, it sorts the matched results in descending order of confidence score and returns the sorted list, so the best matches appear first.


In [22]:
database_addresses = [
    "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081",
    "12B Sai Residency MG Road Hyd 500081",
    "Flat 99C, Sai Residency, MG Rd, Hyderabad 500081",
    "Flat 12B, Lake View Apartments, Gachibowli 500032",
    "Plot 45, Jubilee Hills Hyderabad 500033",
    "3rd Cross, 5th Main, H.No 45, Banjara Hills"
]


In [23]:
query = "Sai Residency Flat 12B MG Road Hyderabad 500081"

results = bulk_match(query, database_addresses)

print(results)


[{'address': 'Flat 12B, Sai Residency, MG Rd, Hyderabad 500081', 'confidence_score': 150.0}, {'address': '12B Sai Residency MG Road Hyd 500081', 'confidence_score': 144.38}]


In [24]:
def jaccard_similarity(text1, text2):
    set1 = set(text1.split())
    set2 = set(text2.split())
    
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    if union == 0:
        return 0
    
    return (intersection / union) * 100


In [ ]:
Jaccard similarity : This tells how much two texts overlap in terms of words.
This function calculates the **Jaccard similarity** between two texts based on word overlap.

It first splits both texts into words and converts them into sets (to remove duplicates), then computes the size of the intersection (common words) and the union (total unique words).

Finally, it returns the similarity as a percentage using the formula:
**(number of common words / total unique words) × 100**, which measures how similar the two texts are based on shared words.


In [25]:
def calculate_similarity_v2(addr1, addr2):
    comp1 = extract_components_v2(addr1)
    comp2 = extract_components_v2(addr2)
    
    text1 = comp1["full_text"]
    text2 = comp2["full_text"]
    
    # Multiple similarity metrics
    token_score = fuzz.token_sort_ratio(text1, text2)
    partial_score = fuzz.partial_ratio(text1, text2)
    jaccard_score = jaccard_similarity(text1, text2)
    
    # Blend scores (weighted average)
    text_score = (
        0.5 * token_score +
        0.3 * partial_score +
        0.2 * jaccard_score
    )
    
    # Bonuses
    house_bonus = 0
    if comp1["house_number"] and comp2["house_number"]:
        if comp1["house_number"] == comp2["house_number"]:
            house_bonus = 20
    
    pin_bonus = 0
    if comp1["pincode"] and comp2["pincode"]:
        if comp1["pincode"] == comp2["pincode"]:
            pin_bonus = 30
    
    final_score = text_score + house_bonus + pin_bonus
    
    return {
        "token_score": token_score,
        "partial_score": partial_score,
        "jaccard_score": jaccard_score,
        "blended_text_score": text_score,
        "house_bonus": house_bonus,
        "pincode_bonus": pin_bonus,
        "final_score": final_score
    }


In [ ]:
First, it compares the full address text using three different similarity metrics:

* `token_sort_ratio` (handles word order changes),
* `partial_ratio` (handles partial matches),
* `jaccard_similarity` (measures word overlap).

These three scores are blended using a weighted average (50% token, 30% partial, 20% Jaccard) to produce a stronger overall text similarity score.

Then, it adds rule-based bonuses:

* +20 points if house numbers match
* +30 points if PIN codes match

Finally, it combines the blended text score with these bonuses to produce the `final_score` and returns a detailed breakdown of all components.


In [26]:
addr1 = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081"
addr2 = "12B Sai Residency MG Road Hyd 500081"

print(calculate_similarity_v2(addr1, addr2))


{'token_score': 94.3820224719101, 'partial_score': 100.0, 'jaccard_score': 87.5, 'blended_text_score': 94.69101123595505, 'house_bonus': 20, 'pincode_bonus': 30, 'final_score': 144.69101123595505}


In [43]:
def decide_match_v3(addr1, addr2, threshold=90):
    result = calculate_similarity_v2(addr1, addr2)
    
    comp1 = extract_components_v2(addr1)
    comp2 = extract_components_v2(addr2)
    
    final_score = result["final_score"]
    
    # Hard PIN conflict
    if comp1["pincode"] and comp2["pincode"]:
        if comp1["pincode"] != comp2["pincode"]:
            return {
                "decision": "NO MATCH",
                "reason": "Different PIN codes",
                "confidence_score": round(final_score, 2)
            }
    
    # House conflict penalty
    if comp1["house_number"] and comp2["house_number"]:
        if comp1["house_number"] != comp2["house_number"]:
            final_score -= 25
    
    if final_score >= threshold:
        decision = "MATCH"
    else:
        decision = "NO MATCH"
    
    return {
        "decision": decision,
        "confidence_score": round(final_score, 2),
        "details": result
    }


In [ ]:
This function decides whether two addresses match using your improved scoring system.

It first calculates the similarity score using `calculate_similarity_v2`, then checks for strict rules:

* If PIN codes exist and are different → immediately **NO MATCH**.
* If house numbers exist and are different → reduce the score by 25 points.

Finally, it compares the adjusted score with the threshold (default 90) and returns **MATCH** or **NO MATCH** along with the confidence score and score details.


In [44]:
import random

def introduce_noise(address):
    words = address.split()
    
    # Randomly shuffle some words
    if random.random() > 0.5:
        random.shuffle(words)
    
    # Randomly remove a word
    if random.random() > 0.7 and len(words) > 3:
        words.pop(random.randint(0, len(words)-1))
    
    return " ".join(words)


In [45]:
base_addresses = [
    "Flat 12B Sai Residency MG Road Hyderabad 500081",
    "Plot 45 Jubilee Hills Hyderabad 500033",
    "Flat 302 Lake View Apartments Gachibowli 500032",
    "House 18A Banjara Hills Hyderabad 500034",
    "Door 77 KPHB Colony Hyderabad 500072"
]


In [46]:
test_pairs = []

# Create true matches (noisy versions)
for addr in base_addresses:
    noisy_version = introduce_noise(addr)
    test_pairs.append((addr, noisy_version, 1))  # 1 = true match

# Create false matches (different addresses)
for i in range(len(base_addresses)-1):
    test_pairs.append((base_addresses[i], base_addresses[i+1], 0))  # 0 = not match


In [ ]:
This code creates a test dataset to evaluate your address matching system.

First loop:
For each base address, it creates a noisy (slightly modified) version and labels it as 1 (true match) — because both addresses refer to the same place.

Second loop:
It pairs each address with the next different address in the list and labels it as 0 (not a match) — creating false match examples for testing accuracy.

In [47]:
def evaluate_model(test_pairs):
    correct = 0
    total = len(test_pairs)
    
    for addr1, addr2, true_label in test_pairs:
        result = decide_match_v3(addr1, addr2)
        prediction = 1 if result["decision"] == "MATCH" else 0
        
        if prediction == true_label:
            correct += 1
    
    accuracy = correct / total
    return accuracy


In [ ]:
This function checks how accurate your address-matching model is.

It loops through each test pair, predicts whether it’s a match using `decide_match_v3`, and compares the prediction with the true label (1 or 0).
If they match, it counts as correct.

Finally, it calculates **accuracy = correct predictions / total test cases** and returns that value.


In [48]:
accuracy = evaluate_model(test_pairs)
print("Model Accuracy:", round(accuracy * 100, 2), "%")


Model Accuracy: 100.0 %


In [49]:
def evaluate_detailed(test_pairs):
    tp = fp = tn = fn = 0
    
    for addr1, addr2, true_label in test_pairs:
        result = decide_match_v3(addr1, addr2)
        prediction = 1 if result["decision"] == "MATCH" else 0
        
        if prediction == 1 and true_label == 1:
            tp += 1
        elif prediction == 1 and true_label == 0:
            fp += 1
        elif prediction == 0 and true_label == 0:
            tn += 1
        elif prediction == 0 and true_label == 1:
            fn += 1
    
    return {
        "True Positives": tp,
        "False Positives": fp,
        "True Negatives": tn,
        "False Negatives": fn
    }
print(evaluate_detailed(test_pairs))


{'True Positives': 5, 'False Positives': 0, 'True Negatives': 4, 'False Negatives': 0}


In [50]:
print(evaluate_detailed(test_pairs))


{'True Positives': 5, 'False Positives': 0, 'True Negatives': 4, 'False Negatives': 0}


In [51]:
def component_score(comp1, comp2):
    score = 0
    
    # PIN weight = 40
    if comp1["pincode"] and comp2["pincode"]:
        if comp1["pincode"] == comp2["pincode"]:
            score += 40
        else:
            return 0  # hard fail
    
    # House weight = 15
    if comp1["house_number"] and comp2["house_number"]:
        if comp1["house_number"] == comp2["house_number"]:
            score += 15
    
    # City weight = 20
    if comp1["city"] and comp2["city"]:
        if comp1["city"].lower() == comp2["city"].lower():
            score += 20
    
    # Locality weight = 15
    if comp1["locality"] and comp2["locality"]:
        if comp1["locality"].lower() == comp2["locality"].lower():
            score += 15
    
    return score


In [ ]:
This function calculates a **structured component-based score** between two addresses.

It gives fixed weights to matching components:

* PIN match → **+40** (if different, return 0 immediately – hard fail)
* House number match → **+15**
* City match → **+20**
* Locality match → **+15**

It adds up all matching component scores and returns the total score.


In [64]:
def decide_match_v5(addr1, addr2, threshold=75):
    result = calculate_similarity_v2(addr1, addr2)
    
    comp1 = extract_components_v3(addr1)
    comp2 = extract_components_v3(addr2)
    
    breakdown = {}
    
    pin1 = comp1.get("pincode")
    pin2 = comp2.get("pincode")
    
    city1 = comp1.get("city")
    city2 = comp2.get("city")
    
    house1 = comp1.get("house_number")
    house2 = comp2.get("house_number")
    
    locality1 = comp1.get("locality")
    locality2 = comp2.get("locality")
    
    # PIN
    if pin1 and pin2:
        if pin1 == pin2:
            breakdown["pin"] = 40
        else:
            return {
                "decision": "NO MATCH",
                "confidence_score": 0,
                "breakdown": {"pin": "conflict"}
            }
    else:
        breakdown["pin"] = 0
    
    # City
    if city1 and city2 and city1.lower() == city2.lower():
        breakdown["city"] = 20
    else:
        breakdown["city"] = 0
    
    # House
    if house1 and house2 and house1 == house2:
        breakdown["house"] = 15
    else:
        breakdown["house"] = 0
    
    # Locality
    if locality1 and locality2 and locality1.lower() == locality2.lower():
        breakdown["locality"] = 15
    else:
        breakdown["locality"] = 0
    
    # Fuzzy
    breakdown["fuzzy"] = result["final_score"] * 0.1
    
    final_score = sum(
        v for v in breakdown.values() if isinstance(v, (int, float))
    )
    
    decision = "MATCH" if final_score >= threshold else "NO MATCH"
    
    return {
        "decision": decision,
        "confidence_score": round(final_score, 2),
        "breakdown": breakdown
    }


In [ ]:
This function decides if two addresses match using a **weighted component-based scoring system**.

It first extracts structured components (PIN, city, house number, locality).

* If PINs exist and are different → immediately **NO MATCH** (hard conflict).
* If they match, it gives fixed scores: PIN = 40, City = 20, House = 15, Locality = 15.
* It also adds 10% of the fuzzy text similarity score.

All scores are added to get `final_score`.
If the score ≥ 75 → **MATCH**, otherwise → **NO MATCH**, and it returns the decision with a detailed score breakdown.


In [53]:
print(evaluate_detailed(test_pairs))


{'True Positives': 5, 'False Positives': 0, 'True Negatives': 4, 'False Negatives': 0}


In [66]:
print(decide_match_v5(addr1, addr2))


{'decision': 'NO MATCH', 'confidence_score': 69.47, 'breakdown': {'pin': 40, 'city': 0, 'house': 15, 'locality': 0, 'fuzzy': 14.469101123595506}}


In [65]:
print(extract_components_v2(addr1))
print(extract_components_v2(addr2))


{'house_number': '12b', 'pincode': '500081', 'full_text': 'flat 12b sai residency mg road hyderabad 500081'}
{'house_number': '12b', 'pincode': '500081', 'full_text': '12b sai residency mg road hyderabad 500081'}


In [68]:
print(extract_components_v3(addr1))


{'pincode': '500081', 'house_number': '12b', 'city': 'hyderabad', 'full_text': 'flat 12b, sai residency, mg rd, hyderabad 500081'}


In [69]:
def extract_components_v4(address):
    import re
    
    address_lower = address.lower()
    components = {}
    
    # --- PINCODE ---
    pin_match = re.search(r"\b\d{6}\b", address_lower)
    if pin_match:
        components["pincode"] = pin_match.group()
    
    # --- HOUSE NUMBER ---
    house_match = re.search(r"\b\d+[a-z]?\b", address_lower)
    if house_match:
        components["house_number"] = house_match.group()
    
    # --- CITY DETECTION ---
    cities = ["hyderabad", "bangalore", "mumbai", "delhi"]
    for city in cities:
        if city in address_lower:
            components["city"] = city
            break
    
    # --- LOCALITY DETECTION ---
    if "city" in components:
        city_index = address_lower.find(components["city"])
        before_city = address_lower[:city_index].strip()
        # Split by commas to get last meaningful segment
        parts = [p.strip() for p in before_city.split(',') if p.strip()]
        if parts:
            components["locality"] = parts[-1]
    
    # --- FULL TEXT ---
    components["full_text"] = address_lower
    
    return components


In [ ]:
This function extracts structured parts from an address string.

It converts the address to lowercase and then:

* Uses regex to find a 6-digit **PIN code**.
* Uses regex to find a **house number** (digits with optional letter like 12A).
* Checks if any predefined cities (Hyderabad, Bangalore, Mumbai, Delhi) are present.
* Extracts the **locality** as the last comma-separated part before the city.
* Stores the full cleaned text as `full_text`.

Finally, it returns all extracted components in a dictionary.


In [70]:
def decide_match_v6(addr1, addr2, threshold=75):
    result = calculate_similarity_v2(addr1, addr2)
    
    comp1 = extract_components_v4(addr1)
    comp2 = extract_components_v4(addr2)
    
    breakdown = {}
    
    # --- PIN ---
    pin1 = comp1.get("pincode")
    pin2 = comp2.get("pincode")
    if pin1 and pin2:
        if pin1 == pin2:
            breakdown["pin"] = 40
        else:
            return {
                "decision": "NO MATCH",
                "confidence_score": 0,
                "breakdown": {"pin": "conflict"}
            }
    else:
        breakdown["pin"] = 0
    
    # --- CITY ---
    city1 = comp1.get("city")
    city2 = comp2.get("city")
    if city1 and city2 and city1.lower() == city2.lower():
        breakdown["city"] = 20
    else:
        breakdown["city"] = 0
    
    # --- HOUSE ---
    house1 = comp1.get("house_number")
    house2 = comp2.get("house_number")
    if house1 and house2 and house1 == house2:
        breakdown["house"] = 15
    else:
        breakdown["house"] = 0
    
    # --- LOCALITY ---
    loc1 = comp1.get("locality")
    loc2 = comp2.get("locality")
    if loc1 and loc2 and loc1.lower() == loc2.lower():
        breakdown["locality"] = 15
    else:
        breakdown["locality"] = 0
    
    # --- FUZZY ---
    breakdown["fuzzy"] = result["final_score"] * 0.1
    
    final_score = sum(v for v in breakdown.values() if isinstance(v, (int, float)))
    
    decision = "MATCH" if final_score >= threshold else "NO MATCH"
    
    return {
        "decision": decision,
        "confidence_score": round(final_score, 2),
        "breakdown": breakdown
    }


In [ ]:
This function decides whether two addresses match using **structured components + fuzzy similarity**.

It first extracts components using `extract_components_v4`, then assigns weights:

* PIN match → **+40** (if different → immediate NO MATCH)
* City match → **+20**
* House number match → **+15**
* Locality match → **+15**
* Fuzzy similarity → 10% of text similarity score

All scores are added to get the final score.
If the score ≥ 75 → **MATCH**, otherwise → **NO MATCH**, and it returns the decision with a detailed breakdown.


In [71]:
addr1 = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081"
addr2 = "12B Sai Residency MG Road Hyderabad 500081"

print(extract_components_v4(addr1))
print(extract_components_v4(addr2))

print(decide_match_v6(addr1, addr2))


{'pincode': '500081', 'house_number': '12b', 'city': 'hyderabad', 'locality': 'mg rd', 'full_text': 'flat 12b, sai residency, mg rd, hyderabad 500081'}
{'pincode': '500081', 'house_number': '12b', 'city': 'hyderabad', 'locality': '12b sai residency mg road', 'full_text': '12b sai residency mg road hyderabad 500081'}
{'decision': 'MATCH', 'confidence_score': 89.47, 'breakdown': {'pin': 40, 'city': 20, 'house': 15, 'locality': 0, 'fuzzy': 14.469101123595506}}


In [72]:
def extract_components_v5(address):
    import re

    address_lower = address.lower()
    components = {}

    # --- PINCODE ---
    pin_match = re.search(r"\b\d{6}\b", address_lower)
    if pin_match:
        components["pincode"] = pin_match.group()

    # --- HOUSE NUMBER ---
    house_match = re.search(r"\b\d+[a-z]?\b", address_lower)
    if house_match:
        components["house_number"] = house_match.group()

    # --- CITY DETECTION ---
    cities = ["hyderabad", "bangalore", "mumbai", "delhi"]
    for city in cities:
        if city in address_lower:
            components["city"] = city
            break

    # --- LOCALITY DETECTION (Improved) ---
    if "city" in components:
        city_index = address_lower.find(components["city"])
        before_city = address_lower[:city_index].strip()
        # Split by commas or spaces
        parts = [p.strip() for p in re.split(r',|\s', before_city) if p.strip()]
        # Remove house numbers (digits)
        parts = [p for p in parts if not re.fullmatch(r'\d+[a-z]?', p)]
        # Take last 2 meaningful words as locality
        if parts:
            components["locality"] = " ".join(parts[-2:])

    # --- FULL TEXT ---
    components["full_text"] = address_lower

    return components


In [ ]:
This version improves how locality is extracted from the address.

It:

* Extracts **PIN code** (6 digits) using regex.
* Extracts **house number** (number with optional letter like 12A).
* Detects **city** from a predefined list.
* For **locality**, it:

  * Takes text before the city,
  * Splits by commas or spaces,
  * Removes house numbers,
  * Picks the last 2 meaningful words as the locality (more accurate than previous version).
* Stores the full lowercase address as `full_text`.

Finally, it returns all extracted components in a dictionary.


In [73]:
addr1 = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081"
addr2 = "12B Sai Residency MG Road Hyderabad 500081"

print(extract_components_v5(addr1))
print(extract_components_v5(addr2))


{'pincode': '500081', 'house_number': '12b', 'city': 'hyderabad', 'locality': 'mg rd', 'full_text': 'flat 12b, sai residency, mg rd, hyderabad 500081'}
{'pincode': '500081', 'house_number': '12b', 'city': 'hyderabad', 'locality': 'mg road', 'full_text': '12b sai residency mg road hyderabad 500081'}


In [4]:
LOCALITY_ABBREVIATIONS = {
    "rd": "road",
    "st": "street",
    "ave": "avenue",
    "blvd": "boulevard",
    "mg": "mg"  # optional if you want to keep initials
}
def normalize_locality(locality):
    words = locality.split()
    normalized_words = []
    for w in words:
        if w in LOCALITY_ABBREVIATIONS:
            normalized_words.append(LOCALITY_ABBREVIATIONS[w])
        else:
            normalized_words.append(w)
    return " ".join(normalized_words)
import re
def extract_components_v5(address):
    address_lower = address.lower()
    components = {}

    # --- PINCODE ---
    pin_match = re.search(r"\b\d{6}\b", address_lower)
    if pin_match:
        components["pincode"] = pin_match.group()

    # --- HOUSE NUMBER ---
    house_match = re.search(r"\b\d+[a-z]?\b", address_lower)
    if house_match:
        components["house_number"] = house_match.group()

    # --- CITY DETECTION ---
    cities = ["hyderabad", "bangalore", "mumbai", "delhi"]
    for city in cities:
        if city in address_lower:
            components["city"] = city
            break

    # --- LOCALITY DETECTION (Improved) ---
    if "city" in components:
        city_index = address_lower.find(components["city"])
        before_city = address_lower[:city_index].strip()
        # Split by commas or spaces
        parts = [p.strip() for p in re.split(r',|\s', before_city) if p.strip()]
        # Remove house numbers (digits)
        parts = [p for p in parts if not re.fullmatch(r'\d+[a-z]?', p)]
        # Take last 2 meaningful words as locality
        if parts:
            components["locality"] = " ".join(parts[-2:])

    # --- Normalize locality ---
    if "locality" in components:
        components["locality"] = normalize_locality(components["locality"])

    # --- FULL TEXT ---
    components["full_text"] = address_lower

    return components



In [ ]:
This code improves locality handling by **standardizing abbreviations** before comparison.

* `LOCALITY_ABBREVIATIONS` maps short forms like **rd → road**, **st → street**, etc.
* `normalize_locality()` replaces abbreviated words with their full forms so “MG Rd” and “MG Road” become the same.

In `extract_components_v5()`:

* It extracts **PIN**, **house number**, and **city** like before.
* It intelligently detects **locality** (last 2 meaningful words before city).
* Then it **normalizes locality** using the abbreviation mapping.
* Finally, it stores the full lowercase address as `full_text`.

This makes your matching more accurate by handling abbreviation variations.


In [5]:
addr1 = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081"
addr2 = "12B Sai Residency MG Road Hyderabad 500081"

comp1 = extract_components_v5(addr1)
comp2 = extract_components_v5(addr2)

print(comp1)
print(comp2)


{'pincode': '500081', 'house_number': '12b', 'city': 'hyderabad', 'locality': 'mg road', 'full_text': 'flat 12b, sai residency, mg rd, hyderabad 500081'}
{'pincode': '500081', 'house_number': '12b', 'city': 'hyderabad', 'locality': 'mg road', 'full_text': '12b sai residency mg road hyderabad 500081'}


In [9]:
!pip install fuzzywuzzy[speedup]


In [17]:
from fuzzywuzzy import fuzz


In [16]:


def calculate_similarity_v2(addr1, addr2):
    """Returns fuzzy token sort similarity for full text."""
    comp1 = extract_components_v5(addr1)
    comp2 = extract_components_v5(addr2)
    
    text_score = fuzz.token_sort_ratio(comp1["full_text"], comp2["full_text"])
    
    return {
        "text_similarity": text_score,
        "final_score": text_score  # keep for now, will weight later
    }

def decide_match_v6(addr1, addr2, threshold=75):
    """Final version of address match scoring."""
    
    # Extract components
    comp1 = extract_components_v5(addr1)
    comp2 = extract_components_v5(addr2)
    
    # Fuzzy similarity
    result = calculate_similarity_v2(addr1, addr2)
    
    breakdown = {}
    
    # PIN
    if comp1.get("pincode") and comp2.get("pincode"):
        if comp1["pincode"] == comp2["pincode"]:
            breakdown["pin"] = 40
        else:
            return {
                "decision": "NO MATCH",
                "confidence_score": 0,
                "breakdown": {"pin": "conflict"}
            }
    else:
        breakdown["pin"] = 0
    
    # CITY
    if comp1.get("city") and comp2.get("city") and comp1["city"].lower() == comp2["city"].lower():
        breakdown["city"] = 20
    else:
        breakdown["city"] = 0
    
    # HOUSE NUMBER
    if comp1.get("house_number") and comp2.get("house_number") and comp1["house_number"] == comp2["house_number"]:
        breakdown["house"] = 15
    else:
        breakdown["house"] = 0
    
    # LOCALITY
    if comp1.get("locality") and comp2.get("locality") and comp1["locality"].lower() == comp2["locality"].lower():
        breakdown["locality"] = 15
    else:
        breakdown["locality"] = 0
    
    # Fuzzy text similarity (10% weight)
    breakdown["fuzzy"] = result["final_score"] * 0.1
    
    # Final score
    final_score = sum(v for v in breakdown.values() if isinstance(v, (int, float)))
    
    decision = "MATCH" if final_score >= threshold else "NO MATCH"
    
    return {
        "decision": decision,
        "confidence_score": round(final_score, 2),
        "breakdown": breakdown
    }


In [ ]:
calculate_similarity_v2

This function calculates fuzzy text similarity between two addresses.
It extracts cleaned components using extract_components_v5() and then uses fuzz.token_sort_ratio() to compare the full text (ignores word order differences).
It returns the text similarity score, which is currently also used as the final_score.

decide_match_v6

This is your final address matching logic.

It:

Extracts structured components (PIN, city, house number, locality).

Calculates fuzzy similarity.

Assigns weighted scores:

PIN match → +40 (if different → immediate NO MATCH)

City match → +20

House number match → +15

Locality match → +15

Fuzzy similarity → 10% of text score

Adds everything to get final_score.

If final_score ≥ 75 → MATCH, otherwise → NO MATCH, and it returns the detailed breakdown.

In [18]:
# Test addresses
addr1 = "Flat 12B, Sai Residency, MG Rd, Hyderabad 500081"
addr2 = "12B Sai Residency MG Road Hyderabad 500081"

# Call the final matcher
result = decide_match_v6(addr1, addr2)

# Print the result
print(result)


{'decision': 'MATCH', 'confidence_score': 98.5, 'breakdown': {'pin': 40, 'city': 20, 'house': 15, 'locality': 15, 'fuzzy': 8.5}}


In [20]:
import pandas as pd

# -------------------------
# Function to match addresses in batch
# -------------------------
def batch_address_match(list1, list2=None, threshold=75):
    """
    Matches addresses in batch.
    
    Parameters:
        list1: list of addresses (mandatory)
        list2: optional second list of addresses (if None, matches all pairs in list1)
        threshold: confidence threshold for MATCH/NO MATCH
    
    Returns:
        DataFrame with columns:
            - addr1
            - addr2
            - decision
            - confidence_score
            - breakdown
    """
    results = []

    if list2 is None:
        list2 = list1  # match within same list

    for i, addr1 in enumerate(list1):
        for j, addr2 in enumerate(list2):
            # Skip identical index if same list
            if list1 is list2 and i == j:
                continue
            
            result = decide_match_v6(addr1, addr2, threshold)
            results.append({
                "addr1": addr1,
                "addr2": addr2,
                "decision": result["decision"],
                "confidence_score": result["confidence_score"],
                "breakdown": result.get("breakdown", {})
            })
    
    return pd.DataFrame(results)

# -------------------------
# Example usage
# -------------------------
addresses = [
    "Flat 12B Sai Residency MG Road Hyderabad 500081",
    "12B Sai Residency MG Road Hyderabad 500081",
    "Plot 45 Jubilee Hills Hyderabad 500033",
    "Flat 302 Lake View Apartments Gachibowli 500032"
]

df_matches = batch_address_match(addresses)
print(df_matches)


C:\Users\DELL\Desktop\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\DELL\Desktop\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


                                              addr1  \
0   Flat 12B Sai Residency MG Road Hyderabad 500081   
1   Flat 12B Sai Residency MG Road Hyderabad 500081   
2   Flat 12B Sai Residency MG Road Hyderabad 500081   
3        12B Sai Residency MG Road Hyderabad 500081   
4        12B Sai Residency MG Road Hyderabad 500081   
5        12B Sai Residency MG Road Hyderabad 500081   
6            Plot 45 Jubilee Hills Hyderabad 500033   
7            Plot 45 Jubilee Hills Hyderabad 500033   
8            Plot 45 Jubilee Hills Hyderabad 500033   
9   Flat 302 Lake View Apartments Gachibowli 500032   
10  Flat 302 Lake View Apartments Gachibowli 500032   
11  Flat 302 Lake View Apartments Gachibowli 500032   

                                              addr2  decision  \
0        12B Sai Residency MG Road Hyderabad 500081     MATCH   
1            Plot 45 Jubilee Hills Hyderabad 500033  NO MATCH   
2   Flat 302 Lake View Apartments Gachibowli 500032  NO MATCH   
3   Flat 12B Sai Residen

In [ ]:
This code performs batch address matching and returns results in a pandas DataFrame.

batch_address_match()

Takes one or two address lists.

If list2 is not provided, it compares all pairs within the same list.

For each pair, it calls decide_match_v6() to get:

MATCH / NO MATCH

Confidence score

Detailed score breakdown

Stores everything in a list and converts it into a DataFrame for easy analysis.

It also avoids comparing an address with itself when matching within the same list.